In [1]:
from bs4 import BeautifulSoup as BS
import requests
from IPython.core.display import HTML
import pandas as pd
import re
import numpy as np
import math

In [2]:
URL = 'https://www.prosportstransactions.com/football/Search/SearchResults.php?Player=&Team=&BeginDate=2019-06-01&EndDate=2020-05-31&ILChkBx=yes&submit=Search&start=0'
url_start = 'https://www.prosportstransactions.com/football/Search/'
response = requests.get(URL)

In [3]:
def soup_up(url_name):
    soup_response = requests.get(url_name)
    soup_parsed = BS(soup_response.content, 'html.parser')
    return soup_parsed
    #last_table = get_table(last_soup)

In [4]:
def remove_dots(moving_df):
    for x in range(len(moving_df.index)):
        if isinstance(moving_df.loc[x+1][2], str):
            if "\u2022" in moving_df.loc[x+1][2]:
                acq_outcome = moving_df.loc[x+1][2].strip("•")
                moving_df.loc[x+1][2] = acq_outcome
    for x in range(len(moving_df.index)):
        if isinstance(moving_df.loc[x+1][3], str):
            if "\u2022" in moving_df.loc[x+1][3]:
                rel_outcome = moving_df.loc[x+1][3].strip("•")
                moving_df.loc[x+1][3] = rel_outcome
    return moving_df

In [5]:
def next_page(current_soup):
    next_link = current_soup.find_all("a")
    url_end = "blank"
    for a in next_link:
        a_state = a.find(string = "Next")
        if a_state == "Next":
            url_end = a.get('href')
    return url_end

In [6]:
def get_table(table_soup):
    injury_table = pd.read_html(str(table_soup.findAll('table', attrs={'class' : 'datatable center'})))[0]
    new_header = injury_table.iloc[0] #grab the first row for the header
    injury_table = injury_table[1:] #take the data less the header row
    injury_table.columns = new_header #set the header row as the df header
    return injury_table

In [7]:
def add_df(big_df,new_df):
    combined_df = big_df.append(new_df)
    return combined_df

In [8]:
def combine_url(url_next):
    url_start = 'https://www.prosportstransactions.com/football/Search/'
    combined_url = url_start + url_next
    return combined_url

In [9]:
def main_loop(started_df, started_soup):
    for x in range(20):
        print(x)
        fresh_soup = soup_up(started_soup)
        new_table = get_table(fresh_soup)
        clean_table = remove_dots(new_table)
        started_df = add_df(started_df,clean_table)
        print(started_df)
        next_url_end = next_page(fresh_soup)
        if next_url_end == 'blank':
            print("breaking")
            break
        started_soup = combine_url(next_url_end)
    return started_df

In [10]:
start_soup = BS(response.content, 'html.parser')

In [11]:
main_df = get_table(start_soup)
clean_df = remove_dots(main_df)
next_url_end = next_page(start_soup)
first_next = combine_url(next_url_end)

In [12]:
last_df = main_loop(clean_df,first_next)

0
0         Date        Team           Acquired  \
1   2019-06-04    Chargers                NaN   
2   2019-06-04       Colts                NaN   
3   2019-06-04     Jaguars     Roderick Young   
4   2019-06-05      Giants                NaN   
5   2019-06-05    Panthers                NaN   
6   2019-06-06    Dolphins       Isaac Asiata   
7   2019-06-06      Giants    Jeremiah Harris   
8   2019-06-06     Raiders      Ryan Yurachek   
9   2019-06-10       Colts                NaN   
10  2019-06-10    Chargers   Trevion Thompson   
11  2019-06-11      Saints                NaN   
12  2019-06-12    Panthers       Mose Frazier   
13  2019-06-13      Giants    Jacob Thieneman   
14  2019-06-18    Dolphins          Rob Rolle   
15  2019-06-26     Falcons    Jeremy Langford   
16  2019-06-28       Bears       Jomon Dotson   
17  2019-07-01    Chargers                NaN   
18  2019-07-03    Dolphins     Connor Hilland   
19  2019-07-12     Jaguars                NaN   
20  2019-07-19    

0         Date      Team         Acquired       Relinquished  \
1   2019-06-04  Chargers              NaN   Trevion Thompson   
2   2019-06-04     Colts              NaN    Javon Patterson   
3   2019-06-04   Jaguars   Roderick Young                NaN   
4   2019-06-05    Giants              NaN    Jeremiah Harris   
5   2019-06-05  Panthers              NaN       Mose Frazier   
..         ...       ...              ...                ...   
21  2019-07-26     49ers              NaN      Erik Magnuson   
22  2019-07-26     49ers              NaN      Garrett Celek   
23  2019-07-26     49ers              NaN    Jerick McKinnon   
24  2019-07-26     49ers              NaN        Jimmie Ward   
25  2019-07-26     49ers              NaN    Weston Richburg   

0                                               Notes  
1                placed on IR with undisclosed injury  
2                  placed on IR with torn ACL in knee  
3                                   activated from IR  
4      

0         Date      Team         Acquired       Relinquished  \
1   2019-06-04  Chargers              NaN   Trevion Thompson   
2   2019-06-04     Colts              NaN    Javon Patterson   
3   2019-06-04   Jaguars   Roderick Young                NaN   
4   2019-06-05    Giants              NaN    Jeremiah Harris   
5   2019-06-05  Panthers              NaN       Mose Frazier   
..         ...       ...              ...                ...   
21  2019-08-13  Seahawks   Tyrone Swoopes                NaN   
22  2019-08-14  Chargers              NaN     Andrew Vollert   
23  2019-08-14    Giants              NaN    Isaiah Searight   
24  2019-08-14  Seahawks    Davante Davis                NaN   
25  2019-08-14  Steelers    Derrek Thomas                NaN   

0                                  Notes  
1   placed on IR with undisclosed injury  
2     placed on IR with torn ACL in knee  
3                      activated from IR  
4   placed on IR with undisclosed injury  
5        placed 

0         Date      Team         Acquired       Relinquished  \
1   2019-06-04  Chargers              NaN   Trevion Thompson   
2   2019-06-04     Colts              NaN    Javon Patterson   
3   2019-06-04   Jaguars   Roderick Young                NaN   
4   2019-06-05    Giants              NaN    Jeremiah Harris   
5   2019-06-05  Panthers              NaN       Mose Frazier   
..         ...       ...              ...                ...   
21  2019-08-31    Ravens              NaN      Fish Smithson   
22  2019-08-31    Ravens              NaN      Kenneth Dixon   
23  2019-08-31    Ravens              NaN   Randin Crecelius   
24  2019-08-31    Ravens              NaN        Tavon Young   
25  2019-08-31  Redskins              NaN    Alex Smith (D.)   

0                                               Notes  
1                placed on IR with undisclosed injury  
2                  placed on IR with torn ACL in knee  
3                                   activated from IR  
4      

In [13]:
last_df

,Date,Team,Acquired,Relinquished,Notes
1,2019-06-04,Chargers,NaN,Trevion Thompson,placed on IR with undisclosed injury
2,2019-06-04,Colts,NaN,Javon Patterson,placed on IR with torn ACL in knee
3,2019-06-04,Jaguars,Roderick Young,NaN,activated from IR
4,2019-06-05,Giants,NaN,Jeremiah Harris,placed on IR with undisclosed injury
5,2019-06-05,Panthers,NaN,Mose Frazier,placed on IR with fractured arm
...,...,...,...,...,...
21,2019-09-04,Chiefs,Michael Hunter,NaN,activated from IR
22,2019-09-04,Cowboys,Codey McElroy,NaN,activated from IR
23,2019-09-04,Dolphins,Quentin Poling,NaN,activated from IR
24,2019-09-04,Texans,B.J. Bello,NaN,activated from IR
